# Parameters:

In [17]:
symbol = 'ETHUSDT'
symbol_ask_bid_price_difference = 0.01
checkPointPriceStepRatio = 0.01
kLinesInterval = '1s'

# Environment:

In [18]:
!pip install pandas
!pip install matplotlib

import datetime as dt

import matplotlib.dates as dates
import matplotlib.pyplot as plot
import pandas as pd

You should consider upgrading via the '/Users/nikita/Documents/projects/things-team/to-the-moon/ttm-analytics/.venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/nikita/Documents/projects/things-team/to-the-moon/ttm-analytics/.venv/bin/python -m pip install --upgrade pip' command.


# K-lines

In [ ]:
from binance.binance_k_line_loader import load_binance_k_lines
from chart.ttm_chart import draw_line_chart

# https://www.binance.com/en/support/faq/how-to-download-historical-market-data-on-binance-5810ae42176b4770b880ce1f14932262
# k_lines = load_binance_k_lines(f"../../../ttm-data/ETHUSDT-1s-2023-03-01.csv")
k_lines = load_binance_k_lines(f"../../../ttm-data/{symbol}-{kLinesInterval}-*.csv")

draw_line_chart(k_lines.open_timestamp_millis, k_lines.open_price, name='Bid Price')
k_lines

# Ticks:

In [ ]:
kLines = k_lines.iterrows()

ticks = []

for kLineIndex, kLine in kLines:
    kLineOpenTimestampMillis = kLine.open_timestamp_millis
    kLineOpenPrice = kLine.open_price

    tickTimestampMillis = kLineOpenTimestampMillis
    tickTimestampSeconds = int(tickTimestampMillis / 1000)
    tickPrice = kLineOpenPrice

    tick = {}
    tick["timestampSeconds"] = tickTimestampSeconds
    tick["bidPrice"] = tickPrice
    tick["askPrice"] = tickPrice + symbol_ask_bid_price_difference

    ticks.append(tick)

ticksDataFrame = pd.DataFrame(ticks)

ticksDataFrame.head(10)

# Ticks price:

In [ ]:
ticksPlotFigure = plot.figure()
ticksPlotAxes = ticksPlotFigure.add_subplot()
ticksPlotAxes.xaxis.set_major_formatter(dates.DateFormatter('%m-%d %H:%M'))
plot.xticks(rotation=60)

tickDateTimes = [dt.datetime.fromtimestamp(timestampSeconds) for timestampSeconds in ticksDataFrame.timestampSeconds]

ticksPlotAxes.plot(tickDateTimes, ticksDataFrame.bidPrice, color='green')


In [ ]:
orders = []

checkPoint = None


def createCheckPoint(tick):
    checkPoint = {}
    checkPoint['tick'] = tick
    checkPoint['priceStep'] = tick['bidPrice'] * checkPointPriceStepRatio
    checkPoint['fallStepAskPrice'] = checkPoint["tick"]["bidPrice"] - checkPoint['priceStep']
    checkPoint['growStepBidPrice'] = checkPoint["tick"]["askPrice"] + checkPoint['priceStep']
    return checkPoint


def createOrder(tick, orderType):
    order = {}
    order['openTimestampSeconds'] = tick['timestampSeconds']
    order['openAskPrice'] = tick['askPrice']
    order['openBidPrice'] = tick['bidPrice']
    order['type'] = orderType
    # print(f"{orderType} order opened")
    return order


def closeOrder(order, tick):
    order['closeTimestampSeconds'] = tick['timestampSeconds']
    order['closeAskPrice'] = tick['askPrice']
    order['closeBidPrice'] = tick['bidPrice']
    if order['type'] == 'grow':
        order['profit'] = order['closeBidPrice'] - order['openAskPrice']
    elif order['type'] == 'fall':
        order['profit'] = order['openBidPrice'] - order['closeAskPrice']
        # print(f"Order closed with profit {order['profit']}")


currentOrder = None

for tick in ticks:
    if (checkPoint == None):
        checkPoint = createCheckPoint(tick)
        # print(f'Init check point: {checkPoint["tick"]["askPrice"]} / {checkPoint["tick"]["bidPrice"]} ({checkPoint["priceStep"]})')
        continue

    growStepDetected = tick['bidPrice'] >= checkPoint['growStepBidPrice']
    if (growStepDetected):
        # print(f"\nGrow step: {tick['bidPrice']} >= {checkPoint['growStepBidPrice']}")
        checkPoint = createCheckPoint(tick)
        # print(f'New check point: {checkPoint["tick"]["askPrice"]} / {checkPoint["tick"]["bidPrice"]} ({checkPoint["priceStep"]})')

    if (growStepDetected and currentOrder != None):
        closeOrder(currentOrder, tick)
        orders.append(currentOrder)
        currentOrder = None

    if (growStepDetected and currentOrder == None):
        currentOrder = createOrder(tick, 'fall')

    fallStepDetected = tick["askPrice"] <= checkPoint['fallStepAskPrice']
    if (fallStepDetected):
        # print(f"\nFall step: {tick['askPrice']} <= {checkPoint['fallStepAskPrice']}")
        checkPoint = createCheckPoint(tick)
        # print(f'New check point: {checkPoint["tick"]["askPrice"]} / {checkPoint["tick"]["bidPrice"]} ({checkPoint["priceStep"]})')

    if (fallStepDetected and currentOrder != None):
        closeOrder(currentOrder, tick)
        orders.append(currentOrder)
        currentOrder = None

    if (fallStepDetected and currentOrder == None):
        currentOrder = createOrder(tick, 'grow')


In [ ]:
ordersDataFrame = pd.DataFrame(orders)

ordersDataFrame.head(46)

In [ ]:
ordersDataFrame.profit

In [ ]:
profits = []

currentProfit = 0

for order in orders:
    profit = {}
    currentProfit = currentProfit + order['profit']
    profit['value'] = currentProfit
    profit['timestampSeconds'] = order['openTimestampSeconds']
    profits.append(profit)

profitsDataFrame = pd.DataFrame(profits)

In [ ]:
profitPlotFigure = plot.figure()
profitPlotAxes = profitPlotFigure.add_subplot()
profitPlotAxes.xaxis.set_major_formatter(dates.DateFormatter('%m-%d %H:%M'))
plot.xticks(rotation=60)

profitDateTimes = [dt.datetime.fromtimestamp(timestampSeconds) for timestampSeconds in
                   profitsDataFrame.timestampSeconds]

profitPlotAxes.plot(profitDateTimes, profitsDataFrame.value, color='green')

In [ ]:
profitsDataFrame.value

In [ ]:
profits[len(profits) - 1]['value']
